# **Introduction:**

This file serves to design and test a custom implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS). This will be trained and evaluated against the ANN designed previously. 

This ANFIS takes the load history, the distance to the task, and the total distance travelled thus far and performs inference about the suitability of a given robot for a task at hand. 

**Date Created:** 13/01/2025

**Date Modified:** 14/01/2025

# **Import Packages:** 

This section imports all the necessary packages for the ANFIS implementation. 

In [31]:
# import packages:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import product

# **Layer Function & Class Definitions:**

Need to define the membership function to be used first:

In [32]:
# triangular membership function:
def triangular_mf(x, params):
    a, b, c = params

    if x <= a or x >= c:
        return 0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b < x < c:
        return (c - x) / (c - b)
    elif x == b:
        return 1
    elif x == c:
        return 1

Now need to define the ANFIS class:

In [33]:
class ANFIS:
    # object constructor:
    def __init__(self, num_inputs, num_mfs, params = None):
        # need to instantiate the object:
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs
        self.memberships = {}
        self.consequents = {}

        # must first assign the antecedent parameters:
        # if custom:
        if params is not None:
            for j in range(self.num_inputs):
                for i in range(self.num_mfs):
                    self.memberships[f'membership_{j+1}_{i+1}'] = params[j, i, :]
            print('params set to other than None')
        # if not custom, randomly initialize them:
        else:
            for j in range(self.num_inputs):
                for i in range(self.num_mfs):
                    a = np.random.uniform(low = -1.0, high = 1.0)
                    b = np.random.uniform(low = -1.0, high = 1.0)
                    c = np.random.uniform(low = -1.0, high = 1.0)

                    params = np.array([a, b, c])
                    self.memberships[f'membership_{j+1}_{i+1}'] = params

        # now must assign the consequent parameters:
        for rule_index in range(1, self.num_rules + 1):
            p = np.random.uniform(low = -1.0, high = 1.0)
            q = np.random.uniform(low = -1.0, high = 1.0)
            r = np.random.uniform(low = -1.0, high = 1.0)
            s = np.random.uniform(low = -1.0, high = 1.0)
            self.consequents[f'rule_{rule_index}_params'] = (p, q, r, s)

        print('model created!')

    # this is a plotting function to verify that the membership functions are correct:

    def plot_membership_functions(self, max_values):
        x1_max, x2_max, x3_max = max_values

        input_ranges = {
            0: np.linspace(0, x1_max, 1000),
            1: np.linspace(0, x2_max, 1000), 
            2: np.linspace(0, x3_max, 1000),
        }

        label_values = {
            0: 'Low',
            1: 'Medium',
            2: 'High'
        }

        # plot mfs for each input:
        for input_index in range(self.num_inputs):
            x_values = input_ranges[input_index]
            plt.figure(figsize=(12, 8))

            # plot each mfs for the selected input:
            for i in range(self.num_mfs):
                params = self.memberships[f'membership_{input_index+1}_{i+1}']
                y_values = [triangular_mf(x, params) for x in x_values]
                plt.plot(x_values, y_values, label = f'{label_values[i]}')
        
                plt.title(f'Membership Functions for Input X{input_index + 1}')
                plt.xlabel('Input Value')
                plt.ylabel('Degree of Membership')
                plt.legend()
                plt.grid(True)
        plt.show()
    
    # this is the first layer within the anfis, the membership layer:

    def membership_layer(self, inputs):

        membership_values = {}

        # for every input j:
        for j in range(self.num_inputs):
            # for every membership function for that input:
            for i in range(self.num_mfs):
                params = self.memberships[f'membership_{j+1}_{i+1}']
                x = inputs[j]
                membership_values[f'membership_{j+1}_{i+1}'] = triangular_mf(x, params)
        
        return membership_values
    
    # this is the second layer within the anfis, the firing strength layer:

    def firing_strength_layer(self, membership_values):

        firing_strengths = {}
        rules = product(range(self.num_mfs), repeat = self.num_inputs)  # pre-generate rule combinations

        for rule_index, combination in enumerate(rules, start = 1):
            w_k = 1.0
            for input_index, mf_index in enumerate(combination):
                w_k *= membership_values[f'membership_{input_index + 1}_{mf_index + 1}']
            firing_strengths[f'rule_{rule_index}'] = w_k

        return firing_strengths
   
    # this is the third layer within the anfis, the normalization layer:

    def normalization_layer(self, firing_strengths):
        normalized_firing_strengths = {}
        total_strength = sum(firing_strengths.values())

        normalized_index = 1
        for value in firing_strengths.values():
            w_bar_k = value / total_strength 
            normalized_firing_strengths[f'normalized_value_{normalized_index}'] = w_bar_k
            normalized_index += 1

        return normalized_firing_strengths
    
    # this is the fourth layer within the anfis, the rule consequent layer:

    def consequent_layer(self, normalized_firing_strengths, inputs):
        w_k_f_k = {}

        for index, (norm_index, w_bar_k) in enumerate(normalized_firing_strengths.items(), start = 1):
            # map the normalized keys to the consequent keys:
            rule_key = f'rule_{index}_params'
            p, q, r, s = self.consequents[rule_key]

            # compute output from this layer:
            output = w_bar_k * (p * inputs[0] + q * inputs[1] + r * inputs[2] + s)
            w_k_f_k[f'output {index}'] = output

        return w_k_f_k

    # this is the fifth layer within the anfis, the output layer:

    def output_layer(self, w_k_f_k):
        return  sum(w_k_f_k.values())

    # all together, the forward pass through the network is given by:

    def forward_pass(self, inputs):
        # need to first pass inputs through the first layer to fuzzify them:
        fuzzified = self.membership_layer(inputs)

        # now we calculate the firing strength of each rule:
        firing_strengths = self.firing_strength_layer(fuzzified)

        # now we normalize these firing strengths:
        normalized_firing_strengths = self.normalization_layer(firing_strengths)

        # determine the rule consequents:
        consequents = self.consequent_layer(normalized_firing_strengths, inputs)

        # overall network output:
        output = self.output_layer(consequents)

        # return to user:
        return output


Define model parameters:

In [34]:
params = np.array([
    [  # Parameters for input 1
        [0, 0, 6],
        [5/6, 5, 55/6],
        [4, 10, 10]
    ],
    [  # Parameters for input 2
        [0, 0 , 15],
        [25/12, 12.5, 275/12],
        [10, 25, 25]
    ],
    [  # Parameters for input 3
        [0, 0, 30],
        [25/6, 25, 275/6],
        [15, 50, 50]
    ]
])

max_values = np.array([10, 25, 50])

inputs are: [ 2  9 21]


Create the model:

In [35]:
model = ANFIS(num_inputs = 3, num_mfs = 3, params = params)

params set to other than None
model created!


Test the forward pass:

In [41]:
inputs = np.array([2, 11, 21])
print(f'inputs are: {inputs}')

model.forward_pass(inputs)

inputs are: [ 2 11 21]


np.float64(-2.844700316518354)